In [0]:
# Databricks notebook source
# ==========================================================
# UNIFICAR ARCHIVOS JSON
# Proyecto: Liga 1 Perú
# Autor: Oscar García Del Águila
# ==========================================================


from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from datetime import datetime
import bootstrap
from util.utils_liga1 import *


# Obtener parámetros de ADF
dbutils.widgets.text("nombre_archivo", "")
dbutils.widgets.text("anio", "")
dbutils.widgets.text("filesystem", "")
dbutils.widgets.text("capa_landing", "")
dbutils.widgets.text("capa_raw", "")
dbutils.widgets.text("rutaBase", "")
dbutils.widgets.text("ModoEjecucion", "")
dbutils.widgets.text("FechaCarga", "")

nombre_archivo = dbutils.widgets.get("nombre_archivo")
anio = dbutils.widgets.get("anio")
filesystem = dbutils.widgets.get("filesystem")
capa_landing = dbutils.widgets.get("capa_landing").strip("/")
capa_raw = dbutils.widgets.get("capa_raw").strip("/")
rutaBase = dbutils.widgets.get("rutaBase").strip("/")
ModoEjecucion = dbutils.widgets.get("ModoEjecucion")
FechaCarga = dbutils.widgets.get("FechaCarga")

# ----------------------------------------------------------
# CONFIGURACIÓN DE ADLS Y SPARK
# ----------------------------------------------------------
spark = SparkSession.builder.getOrCreate()
setup_adls()
dbutils = get_dbutils()

In [0]:
# ----------------------------------------------------------
# RUTAS ORIGEN Y DESTINO
# ----------------------------------------------------------
ruta_json = get_abfss_path(f"{capa_landing}/{anio}/{nombre_archivo}_{anio}.json")

if ModoEjecucion == "HISTORICO":
    ruta_destino = f"{capa_raw}/{rutaBase}/{nombre_archivo}/temp/{anio}/data"
elif ModoEjecucion == "REPROCESO":
    ruta_destino = f"{capa_raw}/{rutaBase}/{nombre_archivo}/{anio}/01/01/data"
else:
    fecha_fmt = FechaCarga.split(" ")[0].replace("-", "/")
    ruta_destino = f"{capa_raw}/{rutaBase}/{nombre_archivo}/{fecha_fmt}/data"

ruta_abfss = get_abfss_path(ruta_destino)

print("===============================================")
print("UNIFICAR JSON")
print("===============================================")
print(f"Archivo origen : {ruta_json}")
print(f"Ruta destino   : {ruta_abfss}")
print("===============================================")

# ----------------------------------------------------------
# LECTURA Y ESCRITURA
# ----------------------------------------------------------
df = spark.read.option("multiLine", True).json(ruta_json)
print(f"Registros leídos: {df.count()}")

# Convertir automáticamente columnas complejas a string JSON
from pyspark.sql import functions as F

for c in df.columns:
    tipo = dict(df.dtypes).get(c, "")
    if tipo.startswith("array") or tipo.startswith("struct"):
        df = df.withColumn(c, F.to_json(c))

df.write.mode("overwrite").parquet(ruta_abfss)

print("Archivo guardado correctamente en formato Parquet.")
